# Face recognition

This notebook uses the CNN model create in "train_faces.ipynb" to classify faces. The result is done in real-time. 

In [1]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image 
import matplotlib.pyplot as plt
%matplotlib inline

import cv2

Using TensorFlow backend.


Load Keras model training in "train_faces.ipynb"

In [2]:
classifier = load_model('face_recog_3faces.h5')

Preprocess the data

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=40,
                                   horizontal_flip = True)

#test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (32, 32),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

#test_set = test_datagen.flow_from_directory('dataset/test',
#                                            target_size = (32, 32),
#                                            batch_size = 16,
#                                            class_mode = 'binary')

Found 1350 images belonging to 3 classes.


Predict new data. We have 3 categories here, so the result is one hot encoder matrix 1x3 (example is [0, 1, 0]). Finding the index of the maximum value and set it to name of the target. This will make us identify the person easier. 

In [4]:
def predict_newdata(data):
    #prediction = ''
    images = image.img_to_array(data)
    images = np.expand_dims(images, axis = 0)
    result = classifier.predict(images)
    training_set.class_indices
    
    # If we have more than 2 category, the result will be one hot encoder matrix. 
    # We have to find the index of the max value.
    #print(result)
    max_pos = np.argmax(result)
    
    if max_pos == 0:
        prediction = 'kum'
    elif max_pos == 1:
        prediction = 'r'
    elif max_pos == 2:
        prediction = 'trin'
        
    return prediction

Run a live feed from webcam. The predict result will shown on the lower right corner of the frame.

In [5]:
def show_webcam(mirror=False):
    #count = 0
    cam = cv2.VideoCapture(1)
    while True:
        ret_val, img = cam.read()
        # Resize image to 32x32 pixels
        test = cv2.resize(img, (32,32))
        predict = predict_newdata(test)

        # Predict result
        #print('Result :',predict_newdata(test))
        if predict == 'trin':
            cv2.putText(img, predict_newdata(test), (400,400), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0))
        elif predict == 'kum':
            cv2.putText(img, predict_newdata(test), (400,400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0))
        elif predict == 'r':
            cv2.putText(img, predict_newdata(test), (400,400), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255))
            
        cv2.imshow('my webcam', img)
        if cv2.waitKey(1) == 27: 
            break  # esc to quit
    cv2.destroyAllWindows()

def main():
    show_webcam(mirror=True)

if __name__ == '__main__':
    main()